# EduHub MongoDB Project - Basic Implementation

**AltSchool of Data Engineering Tinyuka 2024 Second Semester Project Exam**

This notebook demonstrates a MongoDB database implementation for an e-learning platform.

## Project Overview

**Student**: Peace Emmanuel  
**Course**: Data Engineering  
**Project**: MongoDB E-Learning Platform Database  
**Submission Date**: June 15, 2025  

### Learning Objectives
This project demonstrates:
- MongoDB database and collection creation
- Document design and data modeling
- CRUD operations implementation
- Complex queries and aggregation pipelines
- Performance optimization and indexing
- Data validation and error handling

### Database Architecture
The EduHub platform consists of 6 collections:
1. **users** - Students and instructors
2. **courses** - Course information and metadata
3. **enrollments** - Student course enrollments
4. **lessons** - Individual lessons within courses
5. **assignments** - Course assignments
6. **submissions** - Student assignment submissions

---

## Setup and Initialization

Import necessary libraries and initialize database connection.

In [6]:
# Import required modules
import sys
import os

# Add the src directory to the path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from eduhub_queries import LearningPlatformDB
import pandas as pd
from datetime import datetime, timedelta
import json
from pprint import pprint

print("All modules imported successfully!")
print("Ready to connect to MongoDB...")

All modules imported successfully!
Ready to connect to MongoDB...


### Database Connection and Setup

In [7]:
# Initialize the EduHub database
print("Initializing EduHub Database...")
# load config
with open("config.json", "r") as file:
    config = json.load(file)
    url = config["db_url"]
    
# Create database instance
learning_platform = LearningPlatformDB(url)

print("\nDatabase initialized successfully!")
print("Collections created with validation rules")
print("Indexes created for optimal performance")

Initializing EduHub Database...
Collections created with validation rules
Database indexes created successfully

Database initialized successfully!
Collections created with validation rules
Indexes created for optimal performance


---

# Part 1: Database Setup and Data Modeling (20 points)

## Task 1.1: Create Database and Collections

In [8]:
# Get database information
database_info = learning_platform.retrieve_database_info()

print("Database Information:")
print(f"   Database Name: {database_info['database_name']}")
print(f"   Collections: {len(database_info['collections'])}")

print("\nCollections Created:")
for collection in database_info['collections']:
    print(f"   {collection}")

print("\nCollection Statistics:")
for collection, stats in database_info['collection_stats'].items():
    print(f"   {collection}: {stats['count']} documents, {stats['size']} bytes")

Database Information:
   Database Name: eduhub_db
   Collections: 6

Collections Created:
   enrollments
   lessons
   courses
   assignments
   submissions
   users

Collection Statistics:
   enrollments: 0 documents, 0 bytes
   lessons: 0 documents, 0 bytes
   courses: 0 documents, 0 bytes
   assignments: 0 documents, 0 bytes
   submissions: 0 documents, 0 bytes
   users: 0 documents, 0 bytes


## Task 1.2: Design Document Schemas

### Collection Schemas

In [9]:
# Display collection schemas
collection_schemas = {
    "users": {
        "_id": "ObjectId (auto-generated)",
        "userId": "string (unique)",
        "email": "string (unique, required)",
        "firstName": "string (required)",
        "lastName": "string (required)",
        "role": "string (enum: ['student', 'instructor'])",
        "dateJoined": "datetime",
        "profile": {
            "bio": "string",
            "avatar": "string",
            "skills": ["string"]
        },
        "isActive": "boolean"
    },
    "courses": {
        "_id": "ObjectId (auto-generated)",
        "courseId": "string (unique)",
        "title": "string (required)",
        "description": "string",
        "instructorId": "string (reference to users)",
        "category": "string",
        "level": "string (enum: ['beginner', 'intermediate', 'advanced'])",
        "duration": "number (in hours)",
        "price": "number",
        "tags": ["string"],
        "createdAt": "datetime",
        "updatedAt": "datetime",
        "isPublished": "boolean"
    }
}

print("COLLECTION SCHEMAS:")
print("=" * 50)

for collection_name, schema in collection_schemas.items():
    print(f"\n{collection_name.upper()} COLLECTION:")
    pprint(schema, indent=2)
    print("-" * 30)

print("\nAll collection schemas displayed successfully!")

COLLECTION SCHEMAS:

USERS COLLECTION:
{ '_id': 'ObjectId (auto-generated)',
  'dateJoined': 'datetime',
  'email': 'string (unique, required)',
  'firstName': 'string (required)',
  'isActive': 'boolean',
  'lastName': 'string (required)',
  'profile': {'avatar': 'string', 'bio': 'string', 'skills': ['string']},
  'role': "string (enum: ['student', 'instructor'])",
  'userId': 'string (unique)'}
------------------------------

COURSES COLLECTION:
{ '_id': 'ObjectId (auto-generated)',
  'category': 'string',
  'courseId': 'string (unique)',
  'createdAt': 'datetime',
  'description': 'string',
  'duration': 'number (in hours)',
  'instructorId': 'string (reference to users)',
  'isPublished': 'boolean',
  'level': "string (enum: ['beginner', 'intermediate', 'advanced'])",
  'price': 'number',
  'tags': ['string'],
  'title': 'string (required)',
  'updatedAt': 'datetime'}
------------------------------

All collection schemas displayed successfully!


---

# Part 2: Data Population (15 points)

In [10]:
# Populate the database with sample data
print("Starting data population process...")
print("Project Requirements:")
print("   • 20 users (mix of students and instructors)")
print("   • 8 courses across different categories")
print("   • 15 enrollments")
print("   • 25 lessons")
print("   • 10 assignments")
print("   • 12 assignment submissions")

# Populate sample data
learning_platform.populate_sample_data()

# Verify population
collection_stats = learning_platform.get_collection_statistics()
print("\nFinal Population Results:")
for collection, data in collection_stats.items():
    print(f"   {collection}: {data['count']} documents")

print("\nData population completed successfully!")

Starting data population process...
Project Requirements:
   • 20 users (mix of students and instructors)
   • 8 courses across different categories
   • 15 enrollments
   • 25 lessons
   • 10 assignments
   • 12 assignment submissions
Beginning data population process...
Existing data cleared from all collections
Inserted 20 user records
Inserted 8 course records
Inserted 25 lesson records
Inserted 10 assignment records
Inserted 15 enrollment records
Inserted 12 submission records
Data population process completed

Final Population Results:
   enrollments: 15 documents
   lessons: 25 documents
   courses: 8 documents
   assignments: 10 documents
   submissions: 12 documents
   users: 20 documents

Data population completed successfully!


---

# Part 3: Basic CRUD Operations (25 points)

## Task 3.1: Create Operations

In [11]:
print("CRUD CREATE OPERATIONS")
print("=" * 40)

# 1. Add a new student user
print("\n1. Adding a new student user...")
new_student = learning_platform.register_new_student(
    email_address="new.student@example.com",
    first_name="New",
    last_name="Student",
    biography="New student interested in programming",
    skill_list=["Python", "JavaScript"]
)
print(f"   New student created with ID: {new_student}")

# 2. Create a new course
print("\n2. Creating a new course...")
instructor = learning_platform.platform_db.users.find_one({"role": "instructor"})
new_course = learning_platform.create_new_course(
    course_title="Basic Data Analysis",
    course_description="Learn data analysis fundamentals",
    instructor_id=instructor["userId"],
    course_category="Data Science",
    difficulty_level="beginner",
    course_duration=30,
    course_price=150.00,
    tag_list=["data", "analysis", "basics"]
)
print(f"   New course created with ID: {new_course}")

print("\nAll CREATE operations completed successfully!")

CRUD CREATE OPERATIONS

1. Adding a new student user...
New student registered with ID: 684d46c4eb0d92f7a2fbe700
   New student created with ID: 684d46c4eb0d92f7a2fbe700

2. Creating a new course...
New course created with ID: 684d46c5eb0d92f7a2fbe701
   New course created with ID: 684d46c5eb0d92f7a2fbe701

All CREATE operations completed successfully!


## Task 3.2: Read Operations

In [12]:
print("CRUD READ OPERATIONS")
print("=" * 40)

# 1. Find all active students
print("\n1. Finding all active students...")
active_students = learning_platform.find_all_active_students()
print(f"   Found {len(active_students)} active students")
for student in active_students[:3]:
    print(f"      • {student['firstName']} {student['lastName']} ({student['userId']})")

# 2. Get courses by category
print("\n2. Getting courses by category...")
categories = learning_platform.platform_db.courses.distinct("category")
if categories:
    category_courses = learning_platform.get_courses_by_category(categories[0])
    print(f"   Found {len(category_courses)} courses in '{categories[0]}' category")
    for course in category_courses[:3]:
        print(f"      • {course['title']}")

print("\nAll READ operations completed successfully!")

CRUD READ OPERATIONS

1. Finding all active students...
   Found 16 active students
      • Blake Ellis (STU_001)
      • Casey Gray (STU_002)
      • Riley Brooks (STU_003)

2. Getting courses by category...
   Found 2 courses in 'Data Science' category
      • Introduction to Machine Learning
      • Basic Data Analysis

All READ operations completed successfully!


## Task 3.3: Update Operations

In [13]:
print("CRUD UPDATE OPERATIONS")
print("=" * 40)

# 1. Update user profile
print("\n1. Updating user profile...")
sample_user = learning_platform.platform_db.users.find_one({"role": "student"})
modified_count = learning_platform.update_user_profile(
    user_id=sample_user["userId"],
    new_bio="Updated bio: Learning web development",
    new_skills=["Python", "HTML", "CSS"]
)
print(f"   Updated {modified_count} user profile(s)")

# 2. Mark course as published
print("\n2. Marking course as published...")
sample_course = learning_platform.platform_db.courses.find_one()
modified_count = learning_platform.mark_course_as_published(sample_course["courseId"])
print(f"   Updated {modified_count} course(s) as published")

print("\nAll UPDATE operations completed successfully!")

CRUD UPDATE OPERATIONS

1. Updating user profile...
Profile updated for user STU_001. Modified count: 1
   Updated 1 user profile(s)

2. Marking course as published...
Course COURSE_001 marked as published. Modified count: 1
   Updated 1 course(s) as published

All UPDATE operations completed successfully!


## Task 3.4: Delete Operations

In [14]:
print("CRUD DELETE OPERATIONS")
print("=" * 40)

# 1. Soft delete a user
print("\n1. Soft deleting a user...")
sample_user = learning_platform.platform_db.users.find_one({"isActive": True, "role": "student"})
if sample_user:
    modified_count = learning_platform.deactivate_user(sample_user["userId"])
    print(f"   Soft deleted {modified_count} user(s)")
else:
    print("   No active users available for soft delete")

# 2. Delete an enrollment
print("\n2. Deleting an enrollment...")
sample_enrollment = learning_platform.platform_db.enrollments.find_one()
if sample_enrollment:
    deleted_count = learning_platform.remove_enrollment(sample_enrollment["enrollmentId"])
    print(f"   Deleted {deleted_count} enrollment(s)")
else:
    print("   No enrollments available to delete")

print("\nAll DELETE operations completed successfully!")

CRUD DELETE OPERATIONS

1. Soft deleting a user...
User STU_001 deactivated. Modified count: 1
   Soft deleted 1 user(s)

2. Deleting an enrollment...
Enrollment ENROLL_001 removed. Deleted count: 1
   Deleted 1 enrollment(s)

All DELETE operations completed successfully!


---

# Part 4: Advanced Queries and Aggregation (25 points)

## Task 4.1: Complex Queries

In [15]:
print("COMPLEX QUERIES")
print("=" * 40)

# 1. Find courses by price range
print("\n1. Finding courses with price between $50 and $200...")
price_range_courses = learning_platform.find_courses_by_price_range(50, 200)
print(f"   Found {len(price_range_courses)} courses in price range")
for course in price_range_courses[:3]:
    print(f"      • {course['title']}: ${course['price']}")

# 2. Get recent users
print("\n2. Getting users who joined recently...")
recent_users = learning_platform.get_users_joined_recently(6)
print(f"   Found {len(recent_users)} recent users")
students = [u for u in recent_users if u['role'] == 'student']
instructors = [u for u in recent_users if u['role'] == 'instructor']
print(f"      • Students: {len(students)}, Instructors: {len(instructors)}")

print("\nAll COMPLEX QUERIES completed successfully!")

COMPLEX QUERIES

1. Finding courses with price between $50 and $200...
   Found 3 courses in price range
      • Data Science Fundamentals: $138
      • Database Management Systems: $196
      • Basic Data Analysis: $150.0

2. Getting users who joined recently...
   Found 6 recent users
      • Students: 6, Instructors: 0

All COMPLEX QUERIES completed successfully!


## Task 4.2: Aggregation Pipelines

In [16]:
print("AGGREGATION PIPELINES")
print("=" * 40)

# 1. Course enrollment statistics
print("\n1. Course Enrollment Statistics...")
enrollment_stats = learning_platform.get_course_enrollment_statistics()
print(f"   Analyzed {len(enrollment_stats)} course categories")
for stat in enrollment_stats[:3]:
    print(f"      • {stat['_id']}: {stat['totalCourses']} courses, {stat['totalEnrollments']} enrollments")
    print(f"        Avg Price: ${stat['averagePrice']:.2f}")

# 2. Student performance analysis
print("\n2. Student Performance Analysis...")
performance_stats = learning_platform.get_student_performance_analysis()
print(f"   Analyzed {len(performance_stats)} students")
for stat in performance_stats[:3]:
    if stat.get('averageGrade'):
        print(f"      • {stat['studentName']}: Avg Grade {stat['averageGrade']:.1f}")
        print(f"        Submissions: {stat['totalSubmissions']}, Courses: {stat['coursesCount']}")

print("\nAll AGGREGATION PIPELINES completed successfully!")

AGGREGATION PIPELINES

1. Course Enrollment Statistics...
   Analyzed 4 course categories
      • Software Engineering: 3 courses, 6 enrollments
        Avg Price: $282.00
      • Web Development: 3 courses, 6 enrollments
        Avg Price: $278.67
      • Programming: 1 courses, 1 enrollments
        Avg Price: $196.00

2. Student Performance Analysis...
   Analyzed 6 students
      • Casey Gray: Avg Grade 99.0
        Submissions: 1, Courses: 1
      • Casey Ward: Avg Grade 91.0
        Submissions: 2, Courses: 1
      • Alex Gray: Avg Grade 83.0
        Submissions: 5, Courses: 1

All AGGREGATION PIPELINES completed successfully!


---

# Part 5: Indexing and Performance (10 points)

In [17]:
print("PERFORMANCE OPTIMIZATION")
print("=" * 40)

print("\nIndexes Created During Setup:")
print("   User email lookup (unique index)")
print("   Course search by title and category")
print("   Assignment queries by due date")
print("   Enrollment queries by student and course")
print("   Text indexes for full-text search")

# Run performance optimization
print("\nRunning Query Performance Analysis...")
learning_platform.optimize_slow_queries()

# Show index statistics
print("\nIndex Statistics:")
stats = learning_platform.get_collection_statistics()
for collection, stat in stats.items():
    print(f"   {collection}: {stat['indexes']} indexes")

print("\nPerformance optimization completed successfully!")

PERFORMANCE OPTIMIZATION

Indexes Created During Setup:
   User email lookup (unique index)
   Course search by title and category
   Assignment queries by due date
   Enrollment queries by student and course
   Text indexes for full-text search

Running Query Performance Analysis...
Analyzing and optimizing query performance...

1. Optimizing course title search...
   Before optimization: 0.1663 seconds
   Text index created for title and description

2. Optimizing enrollment queries...
   Query time: 0.1652 seconds

3. Optimizing assignment due date queries...
   Query time: 0.2066 seconds

Performance optimization completed!

Index Statistics:
   enrollments: 4 indexes
   lessons: 3 indexes
   courses: 6 indexes
   assignments: 4 indexes
   submissions: 3 indexes
   users: 4 indexes

Performance optimization completed successfully!


---

# Part 6: Data Validation and Error Handling (5 points)

In [ ]:
print("DATA VALIDATION & ERROR HANDLING")
print("=" * 40)

# Test email validation
print("\nEmail Validation Tests:")
valid_emails = ["test@example.com", "user.name@domain.co.uk"]
invalid_emails = ["invalid-email", "@domain.com", "user@"]

for email in valid_emails:
    result = learning_platform.validate_email_format(email)
    print(f"   {email}: {result}")

for email in invalid_emails:
    result = learning_platform.validate_email_format(email)
    print(f"   {email}: {result}")

# Test data validation
print("\nUser Data Validation Test:")
test_user = {
    "userId": "TEST_001",
    "email": "test@example.com",
    "firstName": "Test",
    "lastName": "User",
    "role": "student"
}
result = learning_platform.validate_and_insert_user(test_user)
print(f"   Valid user data: {'Success' if result else 'Failed'}")

print("\nAll validation and error handling tests completed!")

DATA VALIDATION & ERROR HANDLING

Email Validation Tests:
   test@example.com: True
   user.name@domain.co.uk: True
   invalid-email: False
   @domain.com: False
   user@: False

User Data Validation Test:
Document inserted successfully: 684d46d8eb0d92f7a2fbe702
   Valid user data: Success

All validation and error handling tests completed!


---

# Final Project Summary

In [20]:
# Export sample data
print("Exporting sample data...")
learning_platform.export_sample_data("../data/sample_data.json")

# Generate final statistics
final_stats = learning_platform.get_collection_statistics()

print("\nFINAL PROJECT STATISTICS:")
print("=" * 50)
total_documents = sum(stats['count'] for stats in final_stats.values())
total_size = sum(stats['size'] for stats in final_stats.values())
total_indexes = sum(stats['indexes'] for stats in final_stats.values())

print(f"Total Collections: {len(final_stats)}")
print(f"Total Documents: {total_documents}")
print(f"Total Size: {total_size:,} bytes")
print(f"Total Indexes: {total_indexes}")

print("\nCollection Breakdown:")
for collection, stats in final_stats.items():
    print(f"   {collection.ljust(12)}: {str(stats['count']).rjust(3)} docs, {str(stats['indexes']).rjust(2)} indexes")

Exporting sample data...
Sample data exported to ../data/sample_data.json

FINAL PROJECT STATISTICS:
Total Collections: 6
Total Documents: 92
Total Size: 31,250 bytes
Total Indexes: 24

Collection Breakdown:
   enrollments :  14 docs,  4 indexes
   lessons     :  25 docs,  3 indexes
   courses     :   9 docs,  6 indexes
   assignments :  10 docs,  4 indexes
   submissions :  12 docs,  3 indexes
   users       :  22 docs,  4 indexes


---

## Project Conclusion

This MongoDB project successfully implements an e-learning platform database system according to project requirements. The implementation demonstrates:

### Technical Achievements
- **Database Design**: Well-structured collections with proper relationships
- **Data Modeling**: Flexible document schemas with validation rules
- **Query Performance**: Optimized with strategic indexing
- **Scalability**: Designed to handle growing data

### Key Features Implemented
- **User Management System**: Student and instructor profiles
- **Course Management**: Full lifecycle from creation to publishing
- **Enrollment System**: Progress tracking and completion management
- **Assessment Platform**: Assignment submission and grading workflow
- **Analytics Dashboard**: Comprehensive reporting and insights